In [24]:
import json
from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
import numpy as np
import hashlib
from typing import List
import graphviz
from pathlib import Path
import datetime

In [25]:
# Define our embedding class (same as before)
class SimpleHashEmbeddings(Embeddings):
    def __init__(self, dimension: int = 384):
        self.dimension = dimension
    
    def _hash_text(self, text: str) -> List[float]:
        hash_object = hashlib.sha256(text.encode())
        hash_hex = hash_object.hexdigest()
        float_array = []
        for i in range(0, len(hash_hex), 8):
            chunk = hash_hex[i:i+8]
            float_val = int(chunk, 16) / 2**32 - 1
            float_array.append(float_val)
        array = np.array(float_array, dtype=np.float32)
        if len(array) < self.dimension:
            array = np.pad(array, (0, self.dimension - len(array)))
        else:
            array = array[:self.dimension]
        norm = np.linalg.norm(array)
        if norm > 0:
            array = array / norm
        return array.tolist()
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return [self._hash_text(text) for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        return self._hash_text(text)

In [26]:
# Define our documentation prompts
documentation_prompts = {
    'overview': PromptTemplate(
        input_variables=["code"],
        template="""Analyze the following PL/SQL chess engine code and provide a comprehensive overview:

{code}

Please provide:
1. High-level overview of the system
2. Main components and their purposes
3. Key functionalities
4. Design patterns used
5. Important dependencies
"""
    ),
    
    'architecture': PromptTemplate(
        input_variables=["code"],
        template="""Analyze the following PL/SQL chess engine code and describe its architecture:

{code}

Please provide:
1. System architecture overview
2. Component interactions
3. Data flow description
4. Key interfaces
5. Architectural patterns used

Also provide a mermaid diagram showing the architecture.
"""
    ),
    
    'procedure_analysis': PromptTemplate(
        input_variables=["code", "procedure_name"],
        template="""Analyze the following PL/SQL procedure/function in detail:

Procedure/Function: {procedure_name}

Code:
{code}

Please provide:
1. Purpose and functionality
2. Input parameters and their usage
3. Return values and their meaning
4. Algorithm explanation
5. Error handling approach
6. Performance considerations
7. Dependencies and side effects
"""
    ),
    
    'migration_analysis': PromptTemplate(
        input_variables=["code"],
        template="""Analyze the following PL/SQL code and provide a detailed migration approach:

{code}

Please provide:
1. Recommended target technology stack
2. Step-by-step migration plan
3. Potential challenges and solutions
4. Required code transformations
5. Testing strategy
6. Estimated effort and complexity
"""
    )
}

In [27]:
print("Loading vector store and metadata...")

# Initialize embeddings
embeddings = SimpleHashEmbeddings(dimension=384)

try:
    # Load the vector store with allow_dangerous_deserialization set to True
    vectorstore = FAISS.load_local(
        folder_path="../data/vectorstore",
        embeddings=embeddings,
        allow_dangerous_deserialization=True  # Add this parameter
    )
    
    # Load metadata
    metadata_path = Path("../data/metadata/chess_metadata.json")
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)
    else:
        print(f"Warning: Metadata file not found at {metadata_path}")
        metadata = {}

    print("Vector store and metadata loaded successfully!")

except Exception as e:
    print(f"Error loading vector store or metadata: {e}")
    raise

Loading vector store and metadata...
Vector store and metadata loaded successfully!


In [32]:
import os
from dotenv import load_dotenv
import boto3
from botocore.config import Config
from langchain_community.chat_models import BedrockChat
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Load environment variables
load_dotenv()

# Get AWS credentials from environment variables
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')

In [44]:
# Configure boto3 client with retry settings
boto3_config = Config(
    region_name=AWS_REGION,
    retries = {
        'max_attempts': 3,
        'mode': 'standard'
    }
)

def initialize_bedrock_chat():
    """Initialize BedrockChat with appropriate settings"""
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=os.getenv('AWS_REGION', 'us-east-2'),
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
    )
    
    return BedrockChat(
        client=bedrock_client,
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",
        model_kwargs={
            "temperature": 0.7,
            "max_tokens": 4096,
            "top_p": 0.9,
        },
        callbacks=None  # Updated from callback_manager
    )       


In [45]:
def test_chat():
    """Test the chat functionality"""
    try:
        chat = initialize_bedrock_chat()
        
        # Test with a simple message
        response = chat.invoke("Hello! How are you?")
        
        print("\nTest completed successfully!")
        return chat
        
    except Exception as e:
        print(f"Error testing chat: {e}")
        raise

test_chat()


Test completed successfully!


BedrockChat(client=<botocore.client.BedrockRuntime object at 0x12c2dd3d0>, model_id='anthropic.claude-3-sonnet-20240229-v1:0', model_kwargs={'temperature': 0.7, 'max_tokens': 4096, 'top_p': 0.9})

In [46]:
def create_doc_chains(chat_model):
    """Create documentation chains with BedrockChat"""
    chains = {}
    
    # Overview chain
    overview_template = """
    Analyze the following code and provide a comprehensive system overview:
    
    {code}
    
    Focus on:
    1. Main purpose and functionality
    2. Key components and their interactions
    3. Important design patterns and architectural decisions
    4. Integration points
    """
    overview_prompt = ChatPromptTemplate.from_messages([
        HumanMessagePromptTemplate.from_template(overview_template)
    ])
    
    # Architecture chain
    architecture_template = """
    Analyze the following code and provide detailed architectural documentation:
    
    {code}
    
    Include:
    1. System architecture overview
    2. Component relationships
    3. Data flow diagrams (in text form)
    4. Key technical decisions
    5. Scalability considerations
    """
    architecture_prompt = ChatPromptTemplate.from_messages([
        HumanMessagePromptTemplate.from_template(architecture_template)
    ])
    
    # Procedure analysis chain
    procedure_template = """
    Analyze the following procedure and provide detailed documentation:
    
    Procedure: {procedure_name}
    
    Code:
    {code}
    
    Include:
    1. Purpose and functionality
    2. Parameters and return values
    3. Algorithm explanation
    4. Error handling
    5. Performance considerations
    6. Usage examples
    """
    procedure_prompt = ChatPromptTemplate.from_messages([
        HumanMessagePromptTemplate.from_template(procedure_template)
    ])
    
    # Migration analysis chain
    migration_template = """
    Analyze the following system and provide migration recommendations:
    
    {code}
    
    Include:
    1. Modernization opportunities
    2. Potential challenges
    3. Step-by-step migration plan
    4. Risk assessment
    5. Estimated effort
    """
    migration_prompt = ChatPromptTemplate.from_messages([
        HumanMessagePromptTemplate.from_template(migration_template)
    ])
    
    chains['overview'] = LLMChain(llm=chat_model, prompt=overview_prompt)
    chains['architecture'] = LLMChain(llm=chat_model, prompt=architecture_prompt)
    chains['procedure_analysis'] = LLMChain(llm=chat_model, prompt=procedure_prompt)
    chains['migration_analysis'] = LLMChain(llm=chat_model, prompt=migration_prompt)
    
    return chains

In [47]:
def generate_full_documentation(vectorstore, metadata):
    """Generate comprehensive documentation for the chess engine"""
    # Initialize BedrockChat and create chains
    chat_model = initialize_bedrock_chat()
    doc_chains = create_doc_chains(chat_model)
    
    documentation = {
        'generated_at': datetime.datetime.now().isoformat(),
        'overview': {},
        'architecture': {},
        'packages': {},
        'migration_analysis': {}
    }
    
    try:
        print("Generating system overview...")
        # Get overall system context
        system_docs = vectorstore.similarity_search(
            "chess engine system overview architecture main components", k=5)
        combined_system_text = "\n\n".join([doc.page_content for doc in system_docs])
        
        # Generate overview and architecture documentation
        documentation['overview'] = doc_chains['overview'].invoke({"code": combined_system_text})["text"]
        documentation['architecture'] = doc_chains['architecture'].invoke({"code": combined_system_text})["text"]
        
        print("\nAnalyzing individual packages...")
        # Process each package
        for package in metadata['packages']:
            package_name = package['package_name']
            print(f"Processing package: {package_name}")
            
            package_docs = vectorstore.similarity_search(
                f"package {package_name} implementation details", k=3)
            package_text = "\n\n".join([doc.page_content for doc in package_docs])
            
            package_info = {
                'overview': doc_chains['overview'].invoke({"code": package_text})["text"],
                'procedures': {}
            }
            
            # Process procedures
            for routine in package['routines']:
                routine_name = routine['name']
                print(f"  Analyzing procedure: {routine_name}")
                
                routine_docs = vectorstore.similarity_search(
                    f"package {package_name} procedure {routine_name} implementation", k=2)
                routine_text = "\n\n".join([doc.page_content for doc in routine_docs])
                
                package_info['procedures'][routine_name] = \
                    doc_chains['procedure_analysis'].invoke({
                        "code": routine_text,
                        "procedure_name": routine_name
                    })["text"]
            
            documentation['packages'][package_name] = package_info
        
        print("\nGenerating migration analysis...")
        documentation['migration_analysis'] = doc_chains['migration_analysis'].invoke(
            {"code": combined_system_text})["text"]
        
        return documentation
        
    except Exception as e:
        print(f"Error generating documentation: {e}")
        raise

In [42]:
def generate_markdown_documentation(documentation):
    """Convert the JSON documentation to markdown format"""
    md_content = []
    
    # Title and timestamp
    md_content.append("# Chess Engine Documentation")
    md_content.append(f"Generated on: {documentation['generated_at']}\n")
    
    # System Overview
    md_content.append("## System Overview")
    md_content.append(documentation['overview'])
    
    # Architecture
    md_content.append("\n## System Architecture")
    md_content.append(documentation['architecture'])
    
    # Packages
    md_content.append("\n## Packages")
    for package_name, package_info in documentation['packages'].items():
        md_content.append(f"\n### Package: {package_name}")
        md_content.append("#### Overview")
        md_content.append(package_info['overview'])
        
        md_content.append("\n#### Procedures and Functions")
        for proc_name, proc_doc in package_info['procedures'].items():
            md_content.append(f"\n##### {proc_name}")
            md_content.append(proc_doc)
    
    # Migration Analysis
    md_content.append("\n## Migration Analysis")
    md_content.append(documentation['migration_analysis'])
    
    # Save to file
    output_file = f"chess_engine_documentation_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
    with open(output_file, 'w') as f:
        f.write('\n'.join(md_content))
    
    print(f"Markdown documentation saved to {output_file}")
    return output_file


In [ ]:
import datetime
import json
import os
from typing import Dict, Any
from langchain.chat_models import BedrockChat
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain

def create_enhanced_doc_chains(chat_model):
    """Create documentation chains with enhanced prompts"""
    chains = {}
    
    # Enhanced system message
    system_message = SystemMessagePromptTemplate.from_template("""
    You are an expert technical documentation writer specializing in software architecture and code analysis.
    Your documentation should be:
    - Clear and concise yet comprehensive
    - Well-structured with proper headings and sections
    - Rich in technical details while remaining accessible
    - Focused on practical implementation details
    - Include relevant code examples and usage patterns
    """)
    
    # Enhanced overview template
    overview_human = HumanMessagePromptTemplate.from_template("""
    Analyze the following code and provide a comprehensive system overview:

    {code}

    Please structure your response with these sections:

    1. Executive Summary
       - Core purpose and key features
       - Target users/use cases
       - Technical stack overview

    2. System Components
       - Major components and their roles
       - Component interactions
       - Data flow overview

    3. Key Technical Features
       - Architecture patterns used
       - Notable algorithms or processes
       - Performance considerations

    4. Integration Points
       - External system dependencies
       - APIs and interfaces
       - Integration patterns

    5. Technical Considerations
       - Scalability features
       - Security measures
       - Performance optimizations

    Use markdown formatting for better readability.
    """)
    
    # Enhanced architecture template
    architecture_human = HumanMessagePromptTemplate.from_template("""
    Analyze the following code and provide detailed architectural documentation:

    {code}

    Structure your response with these sections:

    1. Architectural Overview
       - Architectural style and patterns
       - System boundaries and context
       - High-level component diagram (in text/ASCII)

    2. Component Design
       - Detailed component breakdown
       - Component responsibilities
       - Interface definitions

    3. Data Architecture
       - Data flow patterns
       - State management
       - Storage patterns

    4. Technical Implementation
       - Technology stack details
       - Framework usage
       - Third-party integrations

    5. Design Decisions
       - Key architectural decisions
       - Trade-offs considered
       - Future scalability considerations

    Use markdown formatting and include ASCII diagrams where applicable.
    """)
    
    # Enhanced procedure template
    procedure_human = HumanMessagePromptTemplate.from_template("""
    Analyze the following procedure and provide detailed documentation:

    Procedure: {procedure_name}
    Code: {code}

    Structure your response with these sections:

    1. Purpose and Overview
       - Main functionality
       - Use cases
       - Role in the system

    2. Technical Specification
       - Input parameters
       - Return values
       - Dependencies
       - Error conditions

    3. Implementation Details
       - Algorithm explanation
       - Key variables and their roles
       - Processing steps
       - Error handling approach

    4. Usage Guide
       - Basic usage example
       - Common patterns
       - Best practices
       - Edge cases

    5. Performance and Optimization
       - Complexity analysis
       - Resource usage
       - Optimization opportunities
       - Caching considerations

    Use markdown code blocks for examples and include performance considerations.
    """)
    
    # Enhanced migration template
    migration_human = HumanMessagePromptTemplate.from_template("""
    Analyze the following system and provide migration recommendations:

    {code}

    Structure your response with these sections:

    1. Migration Overview
       - Current system assessment
       - Migration goals
       - Risk assessment
       - Success criteria

    2. Technical Analysis
       - Legacy components identification
       - Technology stack evaluation
       - Technical debt assessment
       - Modernization opportunities

    3. Migration Strategy
       - Phased approach breakdown
       - Dependencies handling
       - Data migration plan
       - Testing strategy

    4. Implementation Plan
       - Step-by-step migration process
       - Timeline estimates
       - Resource requirements
       - Validation checkpoints

    5. Risk Mitigation
       - Potential challenges
       - Contingency plans
       - Rollback procedures
       - Monitoring strategy

    Use markdown formatting and provide clear action items.
    """)

    # Create chains with enhanced prompts
    chains['overview'] = LLMChain(
        llm=chat_model,
        prompt=ChatPromptTemplate.from_messages([system_message, overview_human])
    )
    chains['architecture'] = LLMChain(
        llm=chat_model,
        prompt=ChatPromptTemplate.from_messages([system_message, architecture_human])
    )
    chains['procedure_analysis'] = LLMChain(
        llm=chat_model,
        prompt=ChatPromptTemplate.from_messages([system_message, procedure_human])
    )
    chains['migration_analysis'] = LLMChain(
        llm=chat_model,
        prompt=ChatPromptTemplate.from_messages([system_message, migration_human])
    )
    
    return chains

def generate_enhanced_markdown(documentation: Dict[str, Any]) -> str:
    """Generate enhanced markdown documentation with better formatting"""
    md_content = []
    
    # Title and metadata
    md_content.extend([
        "# Chess Engine Technical Documentation",
        f"*Generated on: {documentation['generated_at']}*",
        "\n## Table of Contents",
        "1. [System Overview](#system-overview)",
        "2. [System Architecture](#system-architecture)",
        "3. [Package Documentation](#package-documentation)",
        "4. [Migration Analysis](#migration-analysis)",
        "\n---\n"
    ])
    
    # System Overview
    md_content.extend([
        "## System Overview",
        documentation['overview'],
        "\n---\n"
    ])
    
    # Architecture
    md_content.extend([
        "## System Architecture",
        documentation['architecture'],
        "\n---\n"
    ])
    
    # Packages
    md_content.append("## Package Documentation")
    for package_name, package_info in documentation['packages'].items():
        md_content.extend([
            f"\n### {package_name}",
            "\n#### Package Overview",
            package_info['overview'],
            "\n#### Procedures and Functions"
        ])
        
        for proc_name, proc_doc in package_info['procedures'].items():
            md_content.extend([
                f"\n##### `{proc_name}`",
                proc_doc,
                "\n---"
            ])
    
    # Migration Analysis
    md_content.extend([
        "\n## Migration Analysis",
        documentation['migration_analysis'],
        "\n---\n",
        "\n## Additional Information",
        "- This documentation was automatically generated using LangChain and Claude 3",
        "- For updates or issues, please contact the development team",
        f"- Last updated: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    ])
    
    return "\n\n".join(md_content)

def save_enhanced_documentation(documentation: Dict[str, Any], base_dir: str = "documentation_output") -> Dict[str, str]:
    """Save both JSON and Markdown versions of the documentation"""
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = os.path.join(base_dir, timestamp)
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate file paths
    json_file = os.path.join(output_dir, f"chess_engine_documentation_{timestamp}.json")
    md_file = os.path.join(output_dir, f"chess_engine_documentation_{timestamp}.md")
    
    # Save JSON documentation
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(documentation, f, indent=2, ensure_ascii=False)
    
    # Generate and save markdown
    markdown_content = generate_enhanced_markdown(documentation)
    with open(md_file, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    print(f"\nDocumentation saved:")
    print(f"JSON: {json_file}")
    print(f"Markdown: {md_file}")
    
    return {
        'json_file': json_file,
        'markdown_file': md_file,
        'output_dir': output_dir
    }

NameError: name 'ChatPromptTemplate' is not defined

In [ ]:
import datetime
import json
from typing import Dict, Any
import os
from dotenv import load_dotenv
import boto3

# Correct imports from langchain
from langchain.chat_models import BedrockChat  # Updated import path
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain

# Load environment variables
load_dotenv()

def initialize_bedrock_chat():
    """Initialize BedrockChat with appropriate settings"""
    try:
        bedrock_client = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.getenv('AWS_REGION', 'us-east-2'),
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
        )
        
        chat = BedrockChat(
            client=bedrock_client,
            model_id="anthropic.claude-3-sonnet-20240229-v1:0",
            model_kwargs={
                "temperature": 0.7,
                "max_tokens": 4096,
                "top_p": 0.9,
            },
            callbacks=None
        )
        print("BedrockChat initialized successfully!")
        return chat
        
    except Exception as e:
        print(f"Error initializing BedrockChat: {e}")
        raise

def create_doc_chains(chat_model):
    """Create documentation chains with BedrockChat"""
    try:
        chains = {}
        
        # System message for all prompts
        system_message = SystemMessagePromptTemplate.from_template(
            "You are a technical documentation expert. Analyze the provided code and generate detailed documentation."
        )
        
        # Overview chain
        overview_human = HumanMessagePromptTemplate.from_template("""
        Analyze the following code and provide a comprehensive system overview:
        
        {code}
        
        Focus on:
        1. Main purpose and functionality
        2. Key components and their interactions
        3. Important design patterns and architectural decisions
        4. Integration points
        """)
        overview_prompt = ChatPromptTemplate.from_messages([system_message, overview_human])
        
        # Architecture chain
        architecture_human = HumanMessagePromptTemplate.from_template("""
        Analyze the following code and provide detailed architectural documentation:
        
        {code}
        
        Include:
        1. System architecture overview
        2. Component relationships
        3. Data flow diagrams (in text form)
        4. Key technical decisions
        5. Scalability considerations
        """)
        architecture_prompt = ChatPromptTemplate.from_messages([system_message, architecture_human])
        
        # Procedure analysis chain
        procedure_human = HumanMessagePromptTemplate.from_template("""
        Analyze the following procedure and provide detailed documentation:
        
        Procedure: {procedure_name}
        
        Code:
        {code}
        
        Include:
        1. Purpose and functionality
        2. Parameters and return values
        3. Algorithm explanation
        4. Error handling
        5. Performance considerations
        6. Usage examples
        """)
        procedure_prompt = ChatPromptTemplate.from_messages([system_message, procedure_human])
        
        # Migration analysis chain
        migration_human = HumanMessagePromptTemplate.from_template("""
        Analyze the following system and provide migration recommendations:
        
        {code}
        
        Include:
        1. Modernization opportunities
        2. Potential challenges
        3. Step-by-step migration plan
        4. Risk assessment
        5. Estimated effort
        """)
        migration_prompt = ChatPromptTemplate.from_messages([system_message, migration_human])
        
        # Create chains
        chains['overview'] = LLMChain(llm=chat_model, prompt=overview_prompt)
        chains['architecture'] = LLMChain(llm=chat_model, prompt=architecture_prompt)
        chains['procedure_analysis'] = LLMChain(llm=chat_model, prompt=procedure_prompt)
        chains['migration_analysis'] = LLMChain(llm=chat_model, prompt=migration_prompt)
        
        return chains
        
    except Exception as e:
        print(f"Error creating documentation chains: {e}")
        raise

def generate_full_documentation(vectorstore, metadata):
    """Generate comprehensive documentation for the chess engine"""
    try:
        # Initialize BedrockChat and create chains
        chat_model = initialize_bedrock_chat()
        doc_chains = create_doc_chains(chat_model)
        
        documentation = {
            'generated_at': datetime.datetime.now().isoformat(),
            'overview': {},
            'architecture': {},
            'packages': {},
            'migration_analysis': {}
        }
        
        print("Generating system overview...")
        # Get overall system context
        system_docs = vectorstore.similarity_search(
            "chess engine system overview architecture main components", k=5)
        combined_system_text = "\n\n".join([doc.page_content for doc in system_docs])
        
        # Generate overview and architecture documentation
        documentation['overview'] = doc_chains['overview'].invoke({"code": combined_system_text})["text"]
        documentation['architecture'] = doc_chains['architecture'].invoke({"code": combined_system_text})["text"]
        
        print("\nAnalyzing individual packages...")
        # Process each package
        for package in metadata['packages']:
            package_name = package['package_name']
            print(f"Processing package: {package_name}")
            
            package_docs = vectorstore.similarity_search(
                f"package {package_name} implementation details", k=3)
            package_text = "\n\n".join([doc.page_content for doc in package_docs])
            
            package_info = {
                'overview': doc_chains['overview'].invoke({"code": package_text})["text"],
                'procedures': {}
            }
            
            # Process procedures
            for routine in package['routines']:
                routine_name = routine['name']
                print(f"  Analyzing procedure: {routine_name}")
                
                routine_docs = vectorstore.similarity_search(
                    f"package {package_name} procedure {routine_name} implementation", k=2)
                routine_text = "\n\n".join([doc.page_content for doc in routine_docs])
                
                package_info['procedures'][routine_name] = \
                    doc_chains['procedure_analysis'].invoke({
                        "code": routine_text,
                        "procedure_name": routine_name
                    })["text"]
            
            documentation['packages'][package_name] = package_info
        
        print("\nGenerating migration analysis...")
        documentation['migration_analysis'] = doc_chains['migration_analysis'].invoke(
            {"code": combined_system_text})["text"]
        
        return documentation
        
    except Exception as e:
        print(f"Error in documentation generation: {e}")
        raise

def save_documentation(documentation, base_dir="output"):
    """Save documentation to files"""
    try:
        # Create output directory if it doesn't exist
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        output_dir = os.path.join(base_dir, timestamp)
        os.makedirs(output_dir, exist_ok=True)
        
        # Save JSON
        json_file = os.path.join(output_dir, f"chess_engine_documentation_{timestamp}.json")
        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(documentation, f, indent=2, ensure_ascii=False)
        
        print(f"\nDocumentation saved to {json_file}")
        return json_file
        
    except Exception as e:
        print(f"Error saving documentation: {e}")
        raise

if __name__ == "__main__":
    try:
        # Generate documentation
        documentation = generate_full_documentation(vectorstore, metadata)
        
        # Save to file
        output_file = save_documentation(documentation)
        
        print("Documentation generation completed successfully!")
        
    except Exception as e:
        print(f"Error in main execution: {e}")
        raise

BedrockChat initialized successfully!
Generating system overview...

Analyzing individual packages...
Processing package: BODY
  Analyzing procedure: WR
  Analyzing procedure: SET_IN
  Analyzing procedure: MOVETXT
